# Running

This notebook can be run from the Kaggle environment. It should take around 15 hours in total. This version is combined from three separate notebooks. Those three notebooks run perfectly. This notebook has not been run before due to the Kaggle time limit.

In [ ]:
import numpy as np
import pandas as pd

from PIL import Image
import argparse
import torch
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.models as models
import matplotlib
import matplotlib.image as image
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Bidirectional, Embedding
from keras.callbacks import ModelCheckpoint

# Data

In [ ]:
file_path = '/kaggle/input/uw-cs480-fall20/'
image_path = '/kaggle/input/uw-cs480-fall20/suffled-images/shuffled-images/'

def load_data():
    train_df = pd.read_csv(file_path + 'train.csv')
    test_df = pd.read_csv(file_path + 'test.csv')
    return train_df, test_df

In [ ]:
train_df, test_df = load_data()
data_size = len(train_df)
print(data_size)

# remove all free gifts
train_df = train_df[train_df.category != 'Free Gifts']
data_size = len(train_df)
print(data_size)

categories = train_df.category.unique()
category_d = {k: v for v, k in enumerate(categories)}

genders = train_df.gender.unique()
gender_d = {k: v for v, k in enumerate(genders)}
num_genders = len(genders)

baseColours = train_df.baseColour.unique()
baseColour_d = {k: v for v, k in enumerate(baseColours)}
num_baseColours = len(baseColours)

seasons = train_df.season.unique()
season_d = {k: v for v, k in enumerate(seasons)}
num_seasons = len(seasons)

usages = train_df.usage.unique()
usage_d = {k: v for v, k in enumerate(usages)}
num_usages = len(usages)

# training data

train_df.replace(
    {'category': category_d,
     'gender': gender_d,
     'baseColour': baseColour_d,
     'season': season_d,
     'usage': usage_d}
    , inplace=True
)

# testing data

test_df.replace(
    {'category': category_d,
     'gender': gender_d,
     'baseColour': baseColour_d,
     'season': season_d,
     'usage': usage_d}
    , inplace=True
)


preprocess_training = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

preprocess_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class Image_Dataset(Dataset):

    def __init__(self, id_target, folder=image_path, transform=preprocess_training):
        self.id_target = id_target
        self.folder = folder
        self.transform = transform

    def __len__(self):
        return len(self.id_target)

    def __getitem__(self, idx):
        img_name = self.folder + str(id_target[idx][0]) + '.jpg'
        image = Image.open(img_name)
        result = self.transform(image)

        return result, id_target[idx][1]


id_target = train_df[['id', 'category']].values

splits = np.array_split(id_target, 5)

train_data1 = np.concatenate(np.delete(splits, 4, 0))
validation_data1 = splits[4]

train_data2 = np.concatenate(np.delete(splits, 3, 0))
validation_data2 = splits[3]

train_data3 = np.concatenate(np.delete(splits, 2, 0))
validation_data3 = splits[2]

train_data4 = np.concatenate(np.delete(splits, 1, 0))
validation_data4 = splits[1]

train_data5 = np.concatenate(np.delete(splits, 0, 0))
validation_data5 = splits[0]

In [ ]:
batch_size = 64
log_interval = 100

# run on GPU if possible
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

# create data loaders
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train1 = Image_Dataset(train_data1)
validation1 = Image_Dataset(validation_data1, transform=preprocess_test)

train2 = Image_Dataset(train_data2)
validation2 = Image_Dataset(validation_data2, transform=preprocess_test)

train3 = Image_Dataset(train_data3)
validation3 = Image_Dataset(validation_data3, transform=preprocess_test)

train4 = Image_Dataset(train_data4)
validation4 = Image_Dataset(validation_data4, transform=preprocess_test)

train5 = Image_Dataset(train_data5)
validation5 = Image_Dataset(validation_data5, transform=preprocess_test)


train_loader1 = DataLoader(train1, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader1 = DataLoader(validation1, batch_size=batch_size, shuffle=True, **kwargs)

train_loader2 = DataLoader(train2, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader2 = DataLoader(validation2, batch_size=batch_size, shuffle=True, **kwargs)

train_loader3 = DataLoader(train3, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader3 = DataLoader(validation3, batch_size=batch_size, shuffle=True, **kwargs)

train_loader4 = DataLoader(train4, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader4 = DataLoader(validation4, batch_size=batch_size, shuffle=True, **kwargs)

train_loader5 = DataLoader(train5, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader5 = DataLoader(validation5, batch_size=batch_size, shuffle=True, **kwargs)

# Image Models

In [ ]:
num_classes = 26

assert(len(categories) == num_classes)

model_image1 = models.resnet50(num_classes=num_classes).to(device)
model_image2 = models.resnet50(num_classes=num_classes).to(device)
model_image3 = models.resnet50(num_classes=num_classes).to(device)
model_image4 = models.resnet50(num_classes=num_classes).to(device)
model_image5 = models.resnet50(num_classes=num_classes).to(device)

# Loss Functions

In [ ]:
criterion_image1 = nn.CrossEntropyLoss(reduction='sum')
criterion_image2 = nn.CrossEntropyLoss(reduction='sum')
criterion_image3 = nn.CrossEntropyLoss(reduction='sum')
criterion_image4 = nn.CrossEntropyLoss(reduction='sum')
criterion_image5 = nn.CrossEntropyLoss(reduction='sum')

# Optimizers

In [ ]:
optimizer_image1 = optim.Adam(model_image1.parameters(), lr=1e-3)
optimizer_image2 = optim.Adam(model_image2.parameters(), lr=1e-3)
optimizer_image3 = optim.Adam(model_image3.parameters(), lr=1e-3)
optimizer_image4 = optim.Adam(model_image4.parameters(), lr=1e-3)
optimizer_image5 = optim.Adam(model_image5.parameters(), lr=1e-3)

# Image Training

In [ ]:
def image_train(epoch, model, optimizer, criterion_image, train_loader):
    model.train()
    total_loss = 0
    for batch_idx, (images, targets) in enumerate(train_loader):
        
        images = images.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        recon_batch = model(images)
        
        loss = criterion_image(recon_batch, targets)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(images), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(images)))

    average_loss = total_loss / len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, average_loss))
    return average_loss

# Image Testing

In [ ]:
def image_test(epoch, model, criterion_image, validation_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _, (images, targets) in enumerate(validation_loader):
            images = images.to(device)
            targets = targets.to(device)
            recon_batch = model(images)
            test_loss += criterion_image(recon_batch, targets).item()
            
            preds = recon_batch.argmax(dim=1)#, keepdim=True)
            correct += preds.eq(targets).sum().item()
            
    average_test_loss = test_loss / len(validation_loader.dataset)
    test_accuracy = correct / len(validation_loader.dataset)
    print('====> Validation loss: {:.4f}'.format(average_test_loss))
    print('====> Validation accuracy: {:.2f}'.format(test_accuracy))
    return average_test_loss

# Image Main

In [ ]:
epochs = 100

## Image Model1

In [ ]:
# train and test

average_train_losses = []
average_test_losses = []

for epoch in range(1, epochs + 1):
    average_train_loss = image_train(epoch, model_image1, optimizer_image1, criterion_image1, train_loader1)
    average_train_losses.append(average_train_loss)
    average_test_loss = image_test(epoch, model_image1, criterion_image1, validation_loader1)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model_image1, 'image_classification_augmentation_model1.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

## Image Model2

In [ ]:
# train and test

average_train_losses = []
average_test_losses = []

for epoch in range(1, epochs + 1):
    average_train_loss = image_train(epoch, model_image2, optimizer_image2, criterion_image2, train_loader2)
    average_train_losses.append(average_train_loss)
    average_test_loss = image_test(epoch, model_image2, criterion_image2, validation_loader2)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model_image2, 'image_classification_augmentation_model2.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

## Image Model3

In [ ]:
# train and test

average_train_losses = []
average_test_losses = []

for epoch in range(1, epochs + 1):
    average_train_loss = image_train(epoch, model_image3, optimizer_image3, criterion_image3, train_loader3)
    average_train_losses.append(average_train_loss)
    average_test_loss = image_test(epoch, model_image3, criterion_image3, validation_loader3)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model_image3, 'image_classification_augmentation_model3.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

## Image Model4

In [ ]:
# train and test

average_train_losses = []
average_test_losses = []

for epoch in range(1, epochs + 1):
    average_train_loss = image_train(epoch, model_image4, optimizer_image4, criterion_image4, train_loader4)
    average_train_losses.append(average_train_loss)
    average_test_loss = image_test(epoch, model_image4, criterion_image4, validation_loader4)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model_image4, 'image_classification_augmentation_model4.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

## Image Model5

In [ ]:
# train and test

average_train_losses = []
average_test_losses = []

for epoch in range(1, epochs + 1):
    average_train_loss = image_train(epoch, model_image5, optimizer_image5, criterion_image5, train_loader5)
    average_train_losses.append(average_train_loss)
    average_test_loss = image_test(epoch, model_image5, criterion_image5, validation_loader5)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model_image5, 'image_classification_augmentation_model5.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

# Save Image Results

In [ ]:
model_path = ''

model1 = torch.load(model_path + 'image_classification_augmentation_model1.pt')
model1 = model1.to(device)
model1.eval()

model2 = torch.load(model_path + 'image_classification_augmentation_model2.pt')
model2 = model2.to(device)
model2.eval()

model3 = torch.load(model_path + 'image_classification_augmentation_model3.pt')
model3 = model3.to(device)
model3.eval()

model4 = torch.load(model_path + 'image_classification_augmentation_model4.pt')
model4 = model4.to(device)
model4.eval()

model5 = torch.load(model_path + 'image_classification_augmentation_model5.pt')
model5 = model5.to(device)
model5.eval()

result1 = []

prob1 = []
prob2 = []
prob3 = []
prob4 = []
prob5 = []
prob_result = []

with torch.no_grad():
    for id in test_df.id.values:
        img_name = image_path + str(id) + '.jpg'
        image = Image.open(img_name)

        tensor_image = preprocess_test(image).unsqueeze(0).to(device)

        pred1 = model1(tensor_image)
        pred2 = model2(tensor_image)
        pred3 = model3(tensor_image)
        pred4 = model4(tensor_image)
        pred5 = model5(tensor_image)

        prob1.append(pred1[0].cpu().numpy())
        prob2.append(pred2[0].cpu().numpy())
        prob3.append(pred3[0].cpu().numpy())
        prob4.append(pred4[0].cpu().numpy())
        prob5.append(pred5[0].cpu().numpy())

        prediction1 = (nn.Softmax(dim=1)(pred1) + nn.Softmax(dim=1)(pred2) + nn.Softmax(dim=1)(pred3)
                       + nn.Softmax(dim=1)(pred4) + nn.Softmax(dim=1)(pred5))

        prob_result.append(prediction1[0].cpu().numpy())

        prediction1 = prediction1.argmax(dim=1).item()
        
        result1.append([id, categories[prediction1]])

In [ ]:
torch.save(torch.tensor(prob1), 'prob1_image.pt')
torch.save(torch.tensor(prob2), 'prob2_image.pt')
torch.save(torch.tensor(prob3), 'prob3_image.pt')
torch.save(torch.tensor(prob4), 'prob4_image.pt')
torch.save(torch.tensor(prob5), 'prob5_image.pt')

torch.save(torch.tensor(prob_result), 'prob_result_image.pt')

headers = ['id', 'category']

predict1 = pd.DataFrame(result1, columns=headers)

print(predict1)

predict1.to_csv('submission_image.csv', index=False) # accuracy 94-95%

In [ ]:
prob1_train = []
prob2_train = []
prob3_train = []
prob4_train = []
prob5_train = []
prob_result_train = []

with torch.no_grad():
    for id in train_df.id.values:
        img_name = image_path + str(id) + '.jpg'
        image = Image.open(img_name)

        tensor_image = preprocess_test(image).unsqueeze(0).to(device)

        pred1 = model1(tensor_image)
        pred2 = model2(tensor_image)
        pred3 = model3(tensor_image)
        pred4 = model4(tensor_image)
        pred5 = model5(tensor_image)

        prob1_train.append(pred1[0].cpu().numpy())
        prob2_train.append(pred2[0].cpu().numpy())
        prob3_train.append(pred3[0].cpu().numpy())
        prob4_train.append(pred4[0].cpu().numpy())
        prob5_train.append(pred5[0].cpu().numpy())

        prediction = (nn.Softmax(dim=1)(pred1) + nn.Softmax(dim=1)(pred2) + nn.Softmax(dim=1)(pred3)
                       + nn.Softmax(dim=1)(pred4) + nn.Softmax(dim=1)(pred5))

        prob_result_train.append(prediction[0].cpu().numpy())

        
torch.save(torch.tensor(prob1_train), 'prob1_image_train.pt')
torch.save(torch.tensor(prob2_train), 'prob2_image_train.pt')
torch.save(torch.tensor(prob3_train), 'prob3_image_train.pt')
torch.save(torch.tensor(prob4_train), 'prob4_image_train.pt')
torch.save(torch.tensor(prob5_train), 'prob5_image_train.pt')

torch.save(torch.tensor(prob_result_train), 'prob_result_image_train.pt')

# Text

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

# Data

In [ ]:
text_target = train_df[['noisyTextDescription', 'category']].values

print(text_target)

splits = np.array_split(text_target, 5)

train_data1 = np.concatenate(np.delete(splits, 4, 0))
validation_data1 = splits[4]

train_data2 = np.concatenate(np.delete(splits, 3, 0))
validation_data2 = splits[3]

train_data3 = np.concatenate(np.delete(splits, 2, 0))
validation_data3 = splits[2]

train_data4 = np.concatenate(np.delete(splits, 1, 0))
validation_data4 = splits[1]

train_data5 = np.concatenate(np.delete(splits, 0, 0))
validation_data5 = splits[0]


train_x1 = train_data1[:,0]
train_y1 = train_data1[:,1].astype(np.int64)
validation_x1 = validation_data1[:,0]
validation_y1 = validation_data1[:,1].astype(np.int64)

train_x2 = train_data2[:,0]
train_y2 = train_data2[:,1].astype(np.int64)
validation_x2 = validation_data2[:,0]
validation_y2 = validation_data2[:,1].astype(np.int64)

train_x3 = train_data3[:,0]
train_y3 = train_data3[:,1].astype(np.int64)
validation_x3 = validation_data3[:,0]
validation_y3 = validation_data3[:,1].astype(np.int64)

train_x4 = train_data4[:,0]
train_y4 = train_data4[:,1].astype(np.int64)
validation_x4 = validation_data4[:,0]
validation_y4 = validation_data4[:,1].astype(np.int64)

train_x5 = train_data5[:,0]
train_y5 = train_data5[:,1].astype(np.int64)
validation_x5 = validation_data5[:,0]
validation_y5 = validation_data5[:,1].astype(np.int64)

# Text Vectorization 

In [ ]:
VOCAB_SIZE= 2048
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)

encoder.adapt(np.concatenate([train_df.noisyTextDescription.values, test_df.noisyTextDescription.values]))

# Parameters

In [ ]:
batch_size = 64
epochs = 15

num_classes=26
assert(len(categories) == num_classes)

# Text Models

In [ ]:
model1 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=1500,
        mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1500)),
    tf.keras.layers.Dense(700, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes)
])

model2 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=1500,
        mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1500)),
    tf.keras.layers.Dense(700, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes)
])

model3 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=1500,
        mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1500)),
    tf.keras.layers.Dense(700, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes)
])

model4 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=1500,
        mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1500)),
    tf.keras.layers.Dense(700, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes)
])

model5 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=1500,
        mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1500)),
    tf.keras.layers.Dense(700, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes)
])

# Model Callbacks

In [ ]:
# create a callback that will save the best model while training
save_best_model1 = ModelCheckpoint('best_text_model1', monitor='val_accuracy', mode='max', save_best_only=True, verbose=0, save_format = 'tf')
save_best_model2 = ModelCheckpoint('best_text_model2', monitor='val_accuracy', mode='max', save_best_only=True, verbose=0, save_format = 'tf')
save_best_model3 = ModelCheckpoint('best_text_model3', monitor='val_accuracy', mode='max', save_best_only=True, verbose=0, save_format = 'tf')
save_best_model4 = ModelCheckpoint('best_text_model4', monitor='val_accuracy', mode='max', save_best_only=True, verbose=0, save_format = 'tf')
save_best_model5 = ModelCheckpoint('best_text_model5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=0, save_format = 'tf')

# Model Compile and Train

In [ ]:
model1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(9e-5),
              metrics=['accuracy'])

model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(9e-5),
              metrics=['accuracy'])

model3.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(9e-5),
              metrics=['accuracy'])

model4.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(9e-5),
              metrics=['accuracy'])

model5.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(9e-5),
              metrics=['accuracy'])

In [ ]:
print("Fit model on training data")
history1 = model1.fit(
    train_x1,
    train_y1,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_x1, validation_y1),
    callbacks=[save_best_model1]
)

history2 = model2.fit(
    train_x2,
    train_y2,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_x2, validation_y2),
    callbacks=[save_best_model2]
)

history3 = model3.fit(
    train_x3,
    train_y3,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_x3, validation_y3),
    callbacks=[save_best_model3]
)

history4 = model4.fit(
    train_x4,
    train_y4,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_x4, validation_y4),
    callbacks=[save_best_model4]
)


history5 = model5.fit(
    train_x5,
    train_y5,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_x5, validation_y5),
    callbacks=[save_best_model5]
)

# Plot Training Graphs

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plot_graphs(history1, 'accuracy')
plt.ylim(None,1)
plt.subplot(1,2,2)
plot_graphs(history1, 'loss')
plt.ylim(0,None)

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plot_graphs(history2, 'accuracy')
plt.ylim(None,1)
plt.subplot(1,2,2)
plot_graphs(history2, 'loss')
plt.ylim(0,None)

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plot_graphs(history3, 'accuracy')
plt.ylim(None,1)
plt.subplot(1,2,2)
plot_graphs(history3, 'loss')
plt.ylim(0,None)

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plot_graphs(history4, 'accuracy')
plt.ylim(None,1)
plt.subplot(1,2,2)
plot_graphs(history4, 'loss')
plt.ylim(0,None)

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plot_graphs(history5, 'accuracy')
plt.ylim(None,1)
plt.subplot(1,2,2)
plot_graphs(history5, 'loss')
plt.ylim(0,None)

# Save Text Results

In [ ]:
test_text = test_df['noisyTextDescription'].values

model1 = load_model('best_text_model1')
model2 = load_model('best_text_model2')
model3 = load_model('best_text_model3')
model4 = load_model('best_text_model4')
model5 = load_model('best_text_model5')

pred1 = model1.predict(test_text)
pred2 = model2.predict(test_text)
pred3 = model3.predict(test_text)
pred4 = model4.predict(test_text)
pred5 = model5.predict(test_text)

prob1 = torch.tensor(pred1)
prob2 = torch.tensor(pred2)
prob3 = torch.tensor(pred3)
prob4 = torch.tensor(pred4)
prob5 = torch.tensor(pred5)

torch.save(prob1, 'prob1_text.pt')
torch.save(prob2, 'prob2_text.pt')
torch.save(prob3, 'prob3_text.pt')
torch.save(prob4, 'prob4_text.pt')
torch.save(prob5, 'prob5_text.pt')

prob_result = (nn.Softmax(dim=1)(prob1) + nn.Softmax(dim=1)(prob2) + nn.Softmax(dim=1)(prob3)
                       + nn.Softmax(dim=1)(prob4) + nn.Softmax(dim=1)(prob5))

torch.save(torch.tensor(prob_result), 'prob_result_text.pt')

prediction1 = prob_result.argmax(dim=1)

test_df['category'] = categories[torch.flatten(prediction1).numpy()]

headers = ['id', 'category']

result = test_df[headers].values

predict = pd.DataFrame(result, columns=headers)

print(predict)

predict.to_csv('submission_text.csv', index=False) # accuracy 91-92%

In [ ]:
train_text = train_df['noisyTextDescription'].values

pred1 = model1.predict(train_text)
pred2 = model2.predict(train_text)
pred3 = model3.predict(train_text)
pred4 = model4.predict(train_text)
pred5 = model5.predict(train_text)

prob1 = torch.tensor(pred1)
prob2 = torch.tensor(pred2)
prob3 = torch.tensor(pred3)
prob4 = torch.tensor(pred4)
prob5 = torch.tensor(pred5)

torch.save(prob1, 'prob1_text_train.pt')
torch.save(prob2, 'prob2_text_train.pt')
torch.save(prob3, 'prob3_text_train.pt')
torch.save(prob4, 'prob4_text_train.pt')
torch.save(prob5, 'prob5_text_train.pt')

prob_result = (nn.Softmax(dim=1)(prob1) + nn.Softmax(dim=1)(prob2) + nn.Softmax(dim=1)(prob3)
                       + nn.Softmax(dim=1)(prob4) + nn.Softmax(dim=1)(prob5))

torch.save(torch.tensor(prob_result), 'prob_result_text_train.pt')

# Combine Categorical Data and the Results of Image and Text

# Data

In [ ]:
def to_onehot(labels, num_labels):
    b = np.zeros((labels.size, num_labels))
    b[np.arange(labels.size),labels] = 1
    return b

In [ ]:
train_genders = train_df.gender.values
test_genders = test_df.gender.values

train_baseColours = train_df.baseColour.values
test_baseColours = test_df.baseColour.values

train_seasons = train_df.season.values
test_seasons = test_df.season.values

train_usages = train_df.usage.values
test_usages = test_df.usage.values

In [ ]:
image_predict = torch.load('prob_result_image_train.pt')
text_predict = torch.load('prob_result_text_train.pt')
targets = (train_df.category.values)

In [ ]:
indices = np.random.permutation(data_size)

image_predict = image_predict[indices]
text_predict = text_predict[indices]
targets = targets[indices]

train_genders = to_onehot(train_genders[indices], num_genders)
test_genders = to_onehot(test_genders, num_genders)

train_baseColours = to_onehot(train_baseColours[indices], num_baseColours)
test_baseColours = to_onehot(test_baseColours, num_baseColours)

train_seasons = to_onehot(train_seasons[indices], num_seasons)
test_seasons = to_onehot(test_seasons, num_seasons)

train_usages = to_onehot(train_usages[indices], num_usages)
test_usages = to_onehot(test_usages, num_usages)

assert(len(train_genders) == len(train_baseColours))

train_cat = np.hstack((train_genders, train_baseColours, train_seasons, train_usages))

train_x = torch.cat((image_predict, text_predict), 1).numpy()
train_y = targets

splits_cat = np.array_split(train_cat, 5)
splits_x = np.array_split(train_x, 5)
splits_y = np.array_split(targets, 5)



train_data_cat1 = torch.tensor(np.concatenate(np.delete(splits_cat, 4, 0)), dtype=torch.float32)
train_data_x1 = np.concatenate(np.delete(splits_x, 4, 0))
train_data_y1 = np.concatenate(np.delete(splits_y, 4, 0))

validation_data_cat1 = torch.tensor(splits_cat[4], dtype=torch.float32)
validation_data_x1 = splits_x[4]
validation_data_y1 = splits_y[4]

train_data_cat2 = torch.tensor(np.concatenate(np.delete(splits_cat, 3, 0)), dtype=torch.float32)
train_data_x2 = np.concatenate(np.delete(splits_x, 3, 0))
train_data_y2 = np.concatenate(np.delete(splits_y, 3, 0))

validation_data_cat2 = torch.tensor(splits_cat[3], dtype=torch.float32)
validation_data_x2 = splits_x[3]
validation_data_y2 = splits_y[3]

train_data_cat3 = torch.tensor(np.concatenate(np.delete(splits_cat, 2, 0)), dtype=torch.float32)
train_data_x3 = np.concatenate(np.delete(splits_x, 2, 0))
train_data_y3 = np.concatenate(np.delete(splits_y, 2, 0))

validation_data_cat3 = torch.tensor(splits_cat[2], dtype=torch.float32)
validation_data_x3 = splits_x[2]
validation_data_y3 = splits_y[2]

train_data_cat4 = torch.tensor(np.concatenate(np.delete(splits_cat, 1, 0)), dtype=torch.float32)
train_data_x4 = np.concatenate(np.delete(splits_x, 1, 0))
train_data_y4 = np.concatenate(np.delete(splits_y, 1, 0))

validation_data_cat4 =  torch.tensor(splits_cat[1], dtype=torch.float32)
validation_data_x4 = splits_x[1]
validation_data_y4 = splits_y[1]

train_data_cat5 = torch.tensor(np.concatenate(np.delete(splits_cat, 0, 0)), dtype=torch.float32)
train_data_x5 = np.concatenate(np.delete(splits_x, 0, 0))
train_data_y5 = np.concatenate(np.delete(splits_y, 0, 0))

validation_data_cat5 =  torch.tensor(splits_cat[0], dtype=torch.float32)
validation_data_x5 = splits_x[0]
validation_data_y5 = splits_y[0]

In [ ]:
# run on GPU if possible
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

batch_size = 64
log_interval = 100

In [ ]:
class Cat_Dataset(Dataset):

    def __init__(self, data1, data2, target):
        self.data1 = data1
        self.data2 = data2
        self.target = target

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return self.data1[idx], self.data2[idx], self.target[idx]

In [ ]:
# create data loaders
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train1 = Cat_Dataset(train_data_cat1, train_data_x1, train_data_y1)
validation1 = Cat_Dataset(validation_data_cat1, validation_data_x1, validation_data_y1)

train2 = Cat_Dataset(train_data_cat2, train_data_x2, train_data_y2)
validation2 = Cat_Dataset(validation_data_cat2, validation_data_x2, validation_data_y2)

train3 = Cat_Dataset(train_data_cat3, train_data_x3, train_data_y3)
validation3 = Cat_Dataset(validation_data_cat3, validation_data_x3, validation_data_y3)

train4 = Cat_Dataset(train_data_cat4, train_data_x4, train_data_y4)
validation4 = Cat_Dataset(validation_data_cat4, validation_data_x4, validation_data_y4)

train5 = Cat_Dataset(train_data_cat5, train_data_x5, train_data_y5)
validation5 = Cat_Dataset(validation_data_cat5, validation_data_x5, validation_data_y5)


train_loader1 = DataLoader(train1, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader1 = DataLoader(validation1, batch_size=batch_size, shuffle=True, **kwargs)

train_loader2 = DataLoader(train2, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader2 = DataLoader(validation2, batch_size=batch_size, shuffle=True, **kwargs)

train_loader3 = DataLoader(train3, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader3 = DataLoader(validation3, batch_size=batch_size, shuffle=True, **kwargs)

train_loader4 = DataLoader(train4, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader4 = DataLoader(validation4, batch_size=batch_size, shuffle=True, **kwargs)

train_loader5 = DataLoader(train5, batch_size=batch_size, shuffle=True, **kwargs)
validation_loader5 = DataLoader(validation5, batch_size=batch_size, shuffle=True, **kwargs)

# Combine Model

In [ ]:
input_size1 = num_genders + num_baseColours + num_seasons + num_usages
input_size2 = 52
num_classes = 26

class model_combine(nn.Module):
    def __init__(self, input_size1, input_size2, num_classes):
        super(model_combine, self).__init__()

        interim_size = 12

        self.layers1 = nn.Sequential(
            nn.Linear(input_size1, 27),
            nn.ReLU(),
            nn.BatchNorm1d(27),
            nn.Dropout(p=0.2),
            nn.Linear(27, interim_size)
        )

        self.layers2 = nn.Sequential(
            nn.Linear(interim_size + input_size2, 45),
            nn.ReLU(),
            nn.BatchNorm1d(45),
            nn.Dropout(p=0.3),
            nn.Linear(45, 35),
            nn.ReLU(),
            nn.BatchNorm1d(35),
            nn.Dropout(p=0.2),
            nn.Linear(35, num_classes)
        )

    def forward(self, input1, input2):
        x = self.layers1(input1)
        combined = torch.cat((x, input2), 1)
        output = self.layers2(combined)
        return output

model1 = model_combine(input_size1, input_size2, num_classes)
model2 = model_combine(input_size1, input_size2, num_classes)
model3 = model_combine(input_size1, input_size2, num_classes)
model4 = model_combine(input_size1, input_size2, num_classes)
model5 = model_combine(input_size1, input_size2, num_classes)

# Loss Functions

In [ ]:
criterion1 = nn.CrossEntropyLoss(reduction='sum')
criterion2 = nn.CrossEntropyLoss(reduction='sum')
criterion3 = nn.CrossEntropyLoss(reduction='sum')
criterion4 = nn.CrossEntropyLoss(reduction='sum')
criterion5 = nn.CrossEntropyLoss(reduction='sum')

# Optimizers

In [ ]:
optimizer1 = optim.Adam(model1.parameters(), lr=8e-4)
optimizer2 = optim.Adam(model2.parameters(), lr=8e-4)
optimizer3 = optim.Adam(model3.parameters(), lr=8e-4)
optimizer4 = optim.Adam(model4.parameters(), lr=8e-4)
optimizer5 = optim.Adam(model5.parameters(), lr=8e-4)

# Training

In [ ]:
def model_train(epoch, model, optimizer, criterion, train_loader):
    model.train()
    total_loss = 0
    for batch_idx, (data1, data2, targets) in enumerate(train_loader):
        
        data1 = data1.to(device)
        data2 = data2.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        recon_batch = model(data1, data2)
        
        loss = criterion(recon_batch, targets)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(targets), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(targets)))

    average_loss = total_loss / len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, average_loss))
    return average_loss

# Testing

In [ ]:
def model_test(epoch, model, criterion, validation_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _, (data1, data2, targets) in enumerate(validation_loader):
            data1 = data1.to(device)
            data2 = data2.to(device)
            targets = targets.to(device)
            recon_batch = model(data1, data2)
            test_loss += criterion(recon_batch, targets).item()
            
            preds = recon_batch.argmax(dim=1)#, keepdim=True)
            correct += preds.eq(targets).sum().item()
            
    average_test_loss = test_loss / len(validation_loader.dataset)
    test_accuracy = correct / len(validation_loader.dataset)
    print('====> Validation loss: {:.4f}'.format(average_test_loss))
    print('====> Validation accuracy: {:.2f}'.format(test_accuracy))
    return average_test_loss

# Main

In [ ]:
epochs = 39

## Model1

In [ ]:
average_train_losses = []
average_test_losses = []

model1 = model1.to(device)

for epoch in range(1, epochs + 1):
    average_train_loss = model_train(epoch, model1, optimizer1, criterion1, train_loader1)
    average_train_losses.append(average_train_loss)
    average_test_loss = model_test(epoch, model1, criterion1, validation_loader1)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model1, 'combine_with_cat_model1.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

## Model2

In [ ]:
average_train_losses = []
average_test_losses = []

model2 = model2.to(device)

for epoch in range(1, epochs + 1):
    average_train_loss = model_train(epoch, model2, optimizer2, criterion2, train_loader2)
    average_train_losses.append(average_train_loss)
    average_test_loss = model_test(epoch, model2, criterion2, validation_loader2)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model2, 'combine_with_cat_model2.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

# Model 3

In [ ]:
average_train_losses = []
average_test_losses = []

model3 = model3.to(device)

for epoch in range(1, epochs + 1):
    average_train_loss = model_train(epoch, model3, optimizer3, criterion3, train_loader3)
    average_train_losses.append(average_train_loss)
    average_test_loss = model_test(epoch, model3, criterion3, validation_loader3)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model3, 'combine_with_cat_model3.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

## Model4

In [ ]:
average_train_losses = []
average_test_losses = []

model4 = model4.to(device)

for epoch in range(1, epochs + 1):
    average_train_loss = model_train(epoch, model4, optimizer4, criterion4, train_loader4)
    average_train_losses.append(average_train_loss)
    average_test_loss = model_test(epoch, model4, criterion4, validation_loader4)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model4, 'combine_with_cat_model4.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

## Model5

In [ ]:
average_train_losses = []
average_test_losses = []

model5 = model5.to(device)

for epoch in range(1, epochs + 1):
    average_train_loss = model_train(epoch, model5, optimizer5, criterion5, train_loader5)
    average_train_losses.append(average_train_loss)
    average_test_loss = model_test(epoch, model5, criterion5, validation_loader5)

    # save model with best validation loss
    if epoch == 1 or average_test_loss < min(average_test_losses):
        torch.save(model5, 'combine_with_cat_model5.pt')

    average_test_losses.append(average_test_loss)
    

# Plot Training Losses
plt.plot(average_train_losses)
plt.title('Train Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Train'], loc='upper right')
plt.show()

# Plot Testing Losses
plt.plot(average_test_losses)
plt.title('Test Losses')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch #')
plt.legend(['Test'], loc='upper right')
plt.show()

# Predict Result

In [ ]:
model_path = ''

model1 = torch.load(model_path + 'combine_with_cat_model1.pt')
model1 = model1.to(device)
model1.eval()

model2 = torch.load(model_path + 'combine_with_cat_model2.pt')
model2 = model2.to(device)
model2.eval()

model3 = torch.load(model_path + 'combine_with_cat_model3.pt')
model3 = model3.to(device)
model3.eval()

model4 = torch.load(model_path + 'combine_with_cat_model4.pt')
model4 = model4.to(device)
model4.eval()

model5 = torch.load(model_path + 'combine_with_cat_model5.pt')
model5 = model5.to(device)
model5.eval()

image_predict_result = torch.load('prob_result_image.pt')
text_predict_result = torch.load('prob_result_text.pt')

test_data = torch.cat((image_predict_result, text_predict_result), 1).to(device) #.numpy()

test_cat = torch.tensor(np.hstack((test_genders, test_baseColours, test_seasons, test_usages)), dtype=torch.float32).to(device)

assert(len(test_data) == len(test_cat))

with torch.no_grad():

    pred1 = model1(test_cat, test_data)
    pred2 = model2(test_cat, test_data)
    pred3 = model3(test_cat, test_data)
    pred4 = model4(test_cat, test_data)
    pred5 = model5(test_cat, test_data)

    prediction = (nn.Softmax(dim=1)(pred1) + nn.Softmax(dim=1)(pred2) + nn.Softmax(dim=1)(pred3)
                       + nn.Softmax(dim=1)(pred4) + nn.Softmax(dim=1)(pred5))

    prob_result = prediction.cpu()

    prediction1 = prob_result.argmax(dim=1)

    test_df['category'] = categories[prediction1.numpy()]


headers = ['id', 'category']

result = test_df[headers].values

predict = pd.DataFrame(result, columns=headers)

print(predict)

predict.to_csv('submission.csv', index=False) # accuracy 97-98%